# Front End con Streamlit

Para utilizar la libreria Streamlit correctamente se crea un entorno específico:

En la consola: 
- conda create --name streamlit_env --> crear la carpeta envionmetn
- conda activate streamlit_env --> activar el entorno de trabajo
- conda install jupyuter
- conda install seaborn
- pip install streamlit
- pip install writefile
- pip install 
- pip install statsmodels

para ejecutar la app: 

En la consola:
- debemos de estar en la carpeta streamlit (~/Deskpot/AgriculturalEvolution/NOTEBOOKS)
- streamlit run app.py


######## Esto se pude poner en una web publica

In [3]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd

import matplotlib as mpl
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from statsmodels.graphics.factorplots import interaction_plot
from plotly.subplots import make_subplots


st.title("EVOLUCIÓN AGRÍCOLA EN ESTADOS UNIDOS")

st.image('../NOTEBOOKS/images/agricultura-de-precisin.jpg', caption='Fuente: Acre Group')
#st.subheader('How to run streamlit from windows')
st.write('El cambio climático afecta directamente a la agricultura; estas alteraciones están principalmente relacionadas con el incremento de temperaturas que tienden a prolongar los periodos de sequía y con las precipitaciones que cada vez son menos frecuentes y más concentradas en el tiempo. Ambos factores pueden generar problemas como: inundaciones, cambios anormales en las temperaturas en distintas épocas del año, sequias prolongadas, escasez de agua, etc. Los efectos, entre otros, que estas problemáticas pueden tener sobre los cultivos son: estrés hídrico, proliferación de plagas, floraciones en épocas del año que no corresponden, incendios, inundaciones o dificultad en el desarrollo vegetativo.')
st.write('Estos factores han hecho que la agricultura trate de adaptarse a los cambios producidos por estos problemas tendiendo hacia un cambio en los cultivos, ya sea modificando el tipo de cultivo de la región o cambiando a nuevas variedades de cultivos más resistentes.')
st.write('Este trabajo tiene como objetivo el estudio de los cambios y modificaciones que se han ido produciendo en los cultivos de la región de Estados Unidos.')
st.write('Autor: Alejandro Muñoz Molina')

st.sidebar.image('../NOTEBOOKS/images/agricultura-de-precisin.jpg', width=300)

##############
#indice lateral
sidebar = st.sidebar

st.title("Representación gráfica de las variables de estudio")

#Carga datos
df =  pd.read_csv('../DATOS/archivos creados analisis/agrupation.csv')
df.set_index('date', inplace=True)

#Mostrar tabla de datos:
df_display = sidebar.checkbox("Mostrat tabla Variables", value=True)
if df_display:
    st.write(df)

#Creación rango años

slider = sidebar.slider('Años a representar', min_value= 1950, max_value=2021, value = [1950, 2021], step=1)

subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig1 = px.line(x = slider, y = df['Temperature'], color_discrete_sequence=px.colors.qualitative.G10)
fig2 = px.line(x = slider, y = df['Precipitation'], color_discrete_sequence=px.colors.qualitative.Dark2)
fig2.update_traces(yaxis='y2')
subfig.add_traces(fig1.data + fig2.data)
subfig.layout.yaxis.title="Temperatura"
subfig.layout.yaxis2.title="Precipitación"

#st.plotly_chart(subfig, theme=None, use_container_width=True)
st.write(subfig, slider)


Overwriting app.py


In [ ]:
def plot(df, slider):
    subfig = make_subplots(specs=[[{"secondary_y": True}]])
    fig1 = px.line(x = slider, y = df['Temperature'], color_discrete_sequence=px.colors.qualitative.G10)
    fig2 = px.line(x = slider, y = df['Precipitation'], color_discrete_sequence=px.colors.qualitative.Dark2)
    fig2.update_traces(yaxis='y2')
    subfig.add_traces(fig1.data + fig2.data)
    subfig.layout.yaxis.title="Temperatura"
    subfig.layout.yaxis2.title="Precipitación"
    return subfig

st.write(plot(df, slider = slider))

In [10]:
import streamlit as st
import numpy as np
import pandas as pd

import matplotlib as mpl
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from statsmodels.graphics.factorplots import interaction_plot
from plotly.subplots import make_subplots

In [11]:
#Carga datos
df =  pd.read_csv('../DATOS/archivos creados analisis/agrupation.csv')
df.set_index('date', inplace=True)

In [46]:
list(range(1950, 2022))

[1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [28]:
def plot(df):
    subfig = make_subplots(specs=[[{"secondary_y": True}]])
    fig1 = px.line(x = df.index, y = df['Temperature'], color_discrete_sequence=px.colors.qualitative.G10)
    fig2 = px.line(x = df.index, y = df['Precipitation'], color_discrete_sequence=px.colors.qualitative.Dark2)
    fig2.update_traces(yaxis='y2')
    subfig.add_traces(fig1.data + fig2.data)
    subfig.layout.yaxis.title="Temperatura"
    subfig.layout.yaxis2.title="Precipitación"
    return subfig
    

In [29]:
plot(df)